## Deep riccy project

## Colab dependencies

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! tar -zxvf /content/drive/MyDrive/Uni/DeepRiccy/refcocog.tar.gz

In [23]:
#set local path to the data

#this is for colab
#local_path = '/content/refcocog/images/'
#local_annotations = '/content/refcocog/annotations/'

#this is for local
local_path = './refcocog/images/' 
local_annotations = './refcocog/annotations/' 

## Dataset

In [55]:
#import section
import json
import pickle
import torch
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
import matplotlib.pyplot as plt



In [69]:
#class creation for the items
class CocogImage():
    def __init__(self, id, json_file, referece):
        self.id = id
        self.data = json_file['images'][id]
        self.bboxes = []
        self.img_id = self.data['id']

        for i in json_file['annotations']:
            if i['image_id'] == self.img_id:
                self.bboxes.append(i)
                
        self.annotations = []
        
        for i in referece:
            if i['image_id'] == self.id:
                self.annotations.append(i)
    
    def show(self):
        img = Image.open(local_path + self.data['file_name'])
        plt.imshow(img)
        plt.show()

    def get_image(self, transform = transforms.Compose([transforms.PILToTensor()])):
        img = Image.open(local_path + self.data['file_name'])
        return transform(img)
    
    def file_name(self):
        return local_path+self.data['file_name']

    def id(self):
        return self.data['id']

    def width(self):
        return self.data['width']

    def height(self):
        return self.data['height']

    def bbox(self):
        return self.bboxes
    
    def annotations(self):
        return self.annotations

In [70]:
class Coco(Dataset):
  def __init__(self, size):
    super(Coco, self).__init__()
    self.size = size
    #load json file
    with open(local_annotations + 'instances.json', 'r') as f:
      self.data = json.load(f)
    #load references(umd)
    with open(local_annotations + 'refs(umd).p', 'rb') as f:
      self.references = pickle.load(f)
      f.close()
    #load references(google)
    with open(local_annotations + 'refs(google).p', 'rb') as f:
      self.references_google = pickle.load(f)
      f.close()
    
    
  def __getitem__(self, idx):
    if idx >= self.size:
      raise IndexError()
    #import image form json
    image = CocogImage(idx, self.data, self.references)
    return image
    
    return image.get_image(), image.bbox(), image.annotations()

  def __len__(self):
    return self.size


## Model

In [71]:
#test
dataset = Coco(100)

In [72]:
#print the first image
dataset[0].show()

### yolo

In [53]:
#install dependencies
! pip install -qr https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt 


In [74]:
import torch

# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Images
imgs = dataset[0].file_name() 
# Inference
results = model(imgs)

# Results
results.print()
results.save()  # or .show()

results.xyxy[0]  # img1 predictions (tensor)
results.pandas().xyxy[0]  # img1 predictions (pandas)

Using cache found in /Users/pappol/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-4-8 Python-3.10.6 torch-1.13.0 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


FileNotFoundError: [Errno 2] No such file or directory: 'COCO_train2014_000000131074.jpg'